In [1]:
import pandas as pd

# 用户信息
user_info = pd.read_csv('data/member_info_0926.txt', header=None, sep='\t')
user_info.columns = ['uid','gender','visit_freq','u_2_cat_a','u_2_cat_b','u_2_cat_c','u_2_cat_d',
                     'u_2_cat_e','u_multi_cat_a','u_multi_cat_b','u_multi_cat_c','u_multi_cat_d',
                     'u_multi_cat_e','u_salt_value','u_focus_topic','u_like_topic']

In [2]:
answer_info = pd.read_csv('data/answer_info_0926.txt', header=None, sep='\t')

answer_info.columns = ['aid', 'qid', 'uid', 'a_start_date', 'a_sw', 'a_w', 'good', 'recommend', 
                       'round_table', 'picture', 'video', 'num_word', 'num_agree', 'num_cancel_agree',
                       'num_commend', 'num_collect', 'num_thank', 'num_report', 'num_helpless', 'num_disagree']

In [58]:
# 训练数据
train = pd.read_csv('data/invite_info_0926.txt', header=None, sep='\t')
train.columns = ['qid', 'uid', 'i_start_date','answer']
train = pd.merge(train, user_info, how='left', on='uid')
# train = pd.merge(train, question_info, how='left', on='qid')

# 测试数据
test = pd.read_csv('data/invite_info_evaluate_0926.txt', header=None, sep='\t')
test.columns = ['qid', 'uid', 'i_start_date']
test = pd.merge(test, user_info, how='left', on='uid')
# test = pd.merge(test, question_info, how='left', on='qid')

In [59]:
train = pd.merge(train, answer_info, how='left', on=['qid', 'uid'])

In [54]:
drop_list = ['gender','visit_freq','u_2_cat_a','u_2_cat_b','u_2_cat_c','u_2_cat_d',
             'u_2_cat_e','u_multi_cat_a','u_multi_cat_b','u_multi_cat_c','u_multi_cat_d',
             'u_multi_cat_e','u_salt_value','u_focus_topic','u_like_topic']
train_has_answer = train[train['answer']==1].drop(drop_list, axis=1)

In [60]:
user_group = answer_info.groupby('uid')

In [62]:
# 简单的统计，未考虑穿越
sum_feat = ['num_word', 'num_agree', 'num_cancel_agree','num_commend', 'num_collect', 
            'num_thank', 'num_report', 'num_helpless', 'num_disagree']
user_stat_raw = user_group[sum_feat].sum()


In [100]:
user_stat_raw.columns = ['u_total_' + x.split('_')[1] + '_raw' for x in sum_feat]

In [66]:
user_stat_raw[15:20]

,u_total_word,u_total_agree,u_total_cancel,u_total_commend,u_total_collect,u_total_thank,u_total_report,u_total_helpless,u_total_disagree
uid,,,,,,,,,
M1000065584,3083,10,0,2,2,3,0,0,0
M1000067574,0,0,0,0,0,0,0,0,0
M1000071070,467,10,0,2,0,0,0,0,0
M1000076506,58,0,0,2,0,1,0,0,0
M1000084325,62,0,0,0,0,0,0,0,0


In [90]:
# 近一个月用户被邀请次数
u_total_invite = train['uid'].value_counts()
# u_total_invite[:10]

u_total_invite.name = 'u_total_invite_raw'

# 若有的 uid 在 train 里未出现，则 u_total_invite 为 null，补0.
user_stat_raw = user_stat_raw.merge(u_total_invite, left_on='uid', right_index=True, how='left').fillna(0).astype('int64')

In [97]:
user_stat_raw[:5]

,u_total_word,u_total_agree,u_total_cancel,u_total_commend,u_total_collect,u_total_thank,u_total_report,u_total_helpless,u_total_disagree,u_total_invite
uid,,,,,,,,,,
M1000000382,0,0,0,0,0,0,0,0,0,8
M1000000983,122,0,0,0,0,0,0,0,0,14
M1000008978,6,0,0,0,0,0,0,0,0,9
M1000020034,90,0,0,0,0,0,0,0,0,3
M1000022555,73,0,0,0,0,0,0,0,0,5


In [102]:
user_stat_raw.columns

Index(['u_total_word_raw', 'u_total_agree_raw', 'u_total_cancel_raw',
       'u_total_commend_raw', 'u_total_collect_raw', 'u_total_thank_raw',
       'u_total_report_raw', 'u_total_helpless_raw', 'u_total_disagree_raw',
       'u_total_invite_raw'],
      dtype='object')

In [103]:
user_stat_raw.to_csv('feature/user_stat_raw.txt', sep='\t')